# Lesson L2 – Visualizing Word Frequencies 🔍📊  

Welcome to **Lesson L2** of the DeepLearning.AI NLP Specialization remake!  

## What you’ll learn  
* **Build** a frequency dictionary from text  
* **Interpret** what “word frequency” tells us about a corpus  
* **Visualize** frequencies with bar charts & word clouds  
* **Explore** your own texts interactively with a Gradio app  

## Why does this matter?  
Word frequencies are the *hello world* of NLP analysis.  
They underpin language modelling, sentiment analysis, TF‑IDF, and more.  
Before diving into complex models, you should be comfortable asking:  

> *“Which words appear most, and why?”*  

## Roadmap  
1. **Setup & installs** – everything runs in Colab with one click  
2. **Toy example** – three tiny sentences to build intuition  
3. **Real dataset** – 10 k tweets from NLTK  
4. **Visualisations** – bar plots and word clouds  
5. **Gradio playground** – experiment live with your own text  

_👉 Let’s get started!_

In [ ]:
# 🍀 Colab setup – run this first!
!pip -q install nltk wordcloud gradio==4.12.0 --progress-bar off

import nltk, ssl, os
# Ensure SSL context for nltk downloader (occasional Colab issue)
try:
    _create_unverified_https_context = ssl._create_unverified_context
    ssl._create_default_https_context = _create_unverified_https_context
except AttributeError:
    pass

# Download required NLTK resources
for resource in ['stopwords', 'punkt', 'twitter_samples']:
    nltk.download(resource, quiet=True)

# 🔧 Patch for some NLTK versions that look for 'punkt_tab'
try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    try:
        nltk.download('punkt_tab', quiet=True)
    except:
        pass

print('✅ NLTK data ready!')

## 1️⃣ Toy example – build intuition  

We’ll start with **three micro‑sentences** so you can see every step.

In [ ]:
from collections import Counter
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt

toy_sentences = [
    "I love NLP!",
    "NLP loves Python.",
    "Python loves you."
]

# Tokenise & lowercase
tokens = [w.lower() for s in toy_sentences for w in word_tokenize(s)]
freq = Counter(tokens)

print("Frequencies:", freq)

# 📊 bar plot
plt.figure(figsize=(6,3))
plt.bar(freq.keys(), freq.values())
plt.title("Toy example word frequencies")
plt.ylabel("Count")
plt.show()

**What happened?**  
* We tokenised sentences with `word_tokenize`.  
* Lower‑cased everything so “Python” and “python” merge.  
* Counted with `Counter`.  
* Visualised counts.  

With just three lines you already have a **corpus fingerprint**!

## 2️⃣ Utility functions (inline – no `utils.py`)  

We now copy the helper code directly into the notebook so learners **don’t need extra files**.

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

def process_tweet(tweet: str):
    """Clean, tokenize, remove stop‑words & stem."""
    tweet = tweet.lower()
    tweet = re.sub(r'https?://\S+', '', tweet)          # remove urls
    tweet = re.sub(r'[^a-z\s]', '', tweet)              # keep letters & space
    tokens = [stemmer.stem(w) for w in tweet.split() if w not in stop_words]
    return tokens

def build_freqs(tweets, ys):
    """Build (word, label) -> freq dict."""
    freqs = {}
    for y, tweet in zip(ys, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            freqs[pair] = freqs.get(pair, 0) + 1
    return freqs

## 3️⃣ Real‑world dataset – 10 k tweets  

We’ll reuse NLTK’s **twitter_samples** (5 k positive + 5 k negative).

In [ ]:
from nltk.corpus import twitter_samples
import numpy as np

pos_tweets = twitter_samples.strings('positive_tweets.json')
neg_tweets = twitter_samples.strings('negative_tweets.json')

tweets = pos_tweets + neg_tweets
ys = np.append(np.ones(len(pos_tweets)), np.zeros(len(neg_tweets)))

print(f"{len(tweets)} tweets loaded")

freqs = build_freqs(tweets, ys)
print(f"{len(freqs)} (word, label) pairs in frequency dictionary")

## 4️⃣ Visualise top words  

We’ll plot the **20 most common words** irrespective of sentiment,  
then generate separate **word clouds** for positive and negative tweets.

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Aggregate counts ignoring sentiment
from collections import Counter
all_words = Counter()
for (w, y), c in freqs.items():
    all_words[w] += c

top20 = all_words.most_common(20)

# Bar chart
plt.figure(figsize=(10,5))
words, counts = zip(*top20)
plt.bar(words, counts)
plt.xticks(rotation=45, ha='right')
plt.title("Top 20 words in the tweet corpus")
plt.ylabel("Frequency")
plt.show()

# Word clouds
pos_words = {w:c for (w,y),c in freqs.items() if y==1}
neg_words = {w:c for (w,y),c in freqs.items() if y==0}

fig, ax = plt.subplots(1,2, figsize=(14,6))
ax[0].imshow(WordCloud(width=400, height=300).generate_from_frequencies(pos_words))
ax[0].axis('off')
ax[0].set_title('Positive tweets')

ax[1].imshow(WordCloud(width=400, height=300).generate_from_frequencies(neg_words))
ax[1].axis('off')
ax[1].set_title('Negative tweets')
plt.show()

**Interpretation tips:**  
* Common neutral tokens (e.g., “love”) may appear in both clouds.  
* Compare *relative* prominence, not absolute counts.  
* Notice slang or emojis that differ by sentiment – an early cue for building classifiers.

## 5️⃣ Interactive Gradio playground 🎛️  

Paste any text and instantly see its word frequencies.  
Try lyrics, news headlines, or even your own tweet!

In [ ]:
import gradio as gr
from collections import Counter

def freq_dict(text):
    tokens = process_tweet(text)
    return dict(Counter(tokens))

with gr.Blocks() as demo:
    gr.Markdown("""### 🔎 Word‑frequency explorer  
Enter text, press **Run**, and inspect the resulting dictionary.""")
    inp = gr.Textbox(label="Your text", lines=4)
    out = gr.JSON(label="Frequencies")
    inp.submit(freq_dict, inp, out)
    gr.Button("Run").click(freq_dict, inp, out)

# Uncomment the line below when running interactively in Colab
# demo.launch()


---  
🎉 **You’ve visualised word frequencies from scratch!**  
Feel free to tweak the code and explore further.  
Next lesson: we’ll build a **Logistic Regression classifier** using these frequencies as features.